### Wesley Janson and Drew Keller
## STAT 27420 Final Project
# Modeling Code

In [1]:
# Load in relevant packages
# Will need to "pip3 install statsmodels", "pip3 install xgboost"

import pandas as pd
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import xgboost as xgb # This one?
from sklearn.ensemble import GradientBoostingClassifier # Or this one?
from data_utils import prep_features

ModuleNotFoundError: No module named 'statsmodels.miscmodels.ordinal_model'

In [2]:
from load_data import data, categorical_vars, cts_vars, other_vars
# categorical_vars and cts_vars are lists of vars in each category
# other_vars are ID and date variables (categorical_vars + cts_vars + other_vars = all vars)

/Users/drewkeller/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
# loading data from online takes ~20 seconds
# to speed up, save data locally and load from there:
DATA_PATH = '../paper_replication_data/new_data.csv'
data.to_csv(DATA_PATH)  # run this once

In [6]:
data = pd.read_csv(DATA_PATH)

/Users/drewkeller/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (18,20,35,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
prep_features(data)  # prep features for modeling; use regression=True for regression models

In [ ]:
# First model, ordered probit-same as Bachmann et al.
mod_prob = OrderedModel(data['YVAR'],
                        data[['X1', 'X2', 'X3']],
                        distr='probit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

predicted = res_prob.model.predict(res_prob.params, exog=data[['X1', 'X2', 'X3']])
predicted

In [ ]:
# Second model - some sort of XGBoost?